In [50]:
# imports
import torch

# TENSORS

## At its core, PyTorch is a library for processing tensors. A tensor is a number, vector, matrix or any n-dimensional array

In [51]:
# creating tensors with single number

t1=torch.tensor(4)
print(t1)
print("-------------------")
type(t1)
print(t1.dtype)

tensor(4)
-------------------
torch.int64


In [52]:
# lets create more tensors
# vector
t1=torch.tensor([1,2,3,4])
t1

tensor([1, 2, 3, 4])

In [53]:
# matrix
t1=torch.tensor([[1,2,3],[11,22,33],[111,222,333]])
t1


tensor([[  1,   2,   3],
        [ 11,  22,  33],
        [111, 222, 333]])

Tensors can have any number of dimensions, and different lengths along each dimension. We can inspect the length along each dimension using the `.shape` property of a tensor.

In [54]:
t1.shape

torch.Size([3, 3])

## Tensor operations and gradients

We can combine tensors with the usual arithmetic operations. Let's look an example:

In [55]:
x=torch.tensor(3.)
w=torch.tensor(4.,requires_grad=True)
b=torch.tensor(5.,requires_grad=True)
x,w,b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

In [56]:
# Arithmetic operations
y = w * x + b # 3 * 4 + 5 = 17
y

tensor(17., grad_fn=<AddBackward0>)

# What makes PyTorch special is that we can automatically compute the derivative of y w.r.t. the tensors that have requires_grad set to True i.e. w and b. To compute the derivatives, we can call the .backward method on our result y.

In [57]:
# compute derivatives
y.backward()

The derivates of `y` w.r.t the input tensors are stored in the `.grad` property of the respective tensors.

In [58]:
# Display gradients
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


As expected, `dy/dw` has the same value as `x` i.e. `3`, and `dy/db` has the value `1`. Note that `x.grad` is `None`, because `x` doesn't have `requires_grad` set to `True`.

The "grad" in `w.grad` stands for gradient, which is another term for derivative, used mainly when dealing with matrices.

# ## Interoperability with Numpy

[Numpy](http://www.numpy.org/) is a popular open source library used for mathematical and scientific computing in Python. It enables efficient operations on large multi-dimensional arrays, and has a large ecosystem of supporting libraries:

* [Matplotlib](https://matplotlib.org/) for plotting and visualization
* [OpenCV](https://opencv.org/) for image and video processing
* [Pandas](https://pandas.pydata.org/) for file I/O and data analysis

Instead of reinventing the wheel, PyTorch interoperates really well with Numpy to leverage its existing ecosystem of tools and libraries.

In [59]:
import numpy as np
x=np.array([[1,2],[3,4]])
x

array([[1, 2],
       [3, 4]])

In [60]:
# We can convert a Numpy array to a PyTorch tensor using `torch.from_numpy`.

y=torch.from_numpy(x)
y

tensor([[1, 2],
        [3, 4]])

In [61]:
# Let's verify that the numpy array and torch tensor have similar data types.
x.dtype, y.dtype

(dtype('int64'), torch.int64)

# Let's Try solving a Linear Regression example with the pytorch

in this tutorial, we'll discuss one of the foundational algorithms in machine learning: Linear regression. We'll create a model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall, and humidity (input variables or features) in a region.

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

- yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1


- yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [62]:
import torch.nn as nn

In [63]:
# the inputs and targets and matrices.

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [64]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [65]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

## We are using 15 training examples to illustrate how to work with large datasets in small batches.

# Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [66]:
from torch.utils.data import TensorDataset

In [67]:
# define the dataset
train_dataset=TensorDataset(inputs,targets)

In [68]:
train_dataset[0:4]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.]]))

### The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:4]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

# We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [69]:
from torch.utils.data import DataLoader

In [70]:
 # define the dataloader
 batchsize=5
 train_dataloader=DataLoader(train_dataset,batchsize,shuffle=True)



### We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.
We can use the data loader in a `for` loop. Let's look at an example.

In [71]:
for inp,tar in train_dataloader:
  print(inp)
  print(tar)
  break

tensor([[ 68.,  96.,  71.],
        [ 87., 134.,  58.],
        [ 73.,  66.,  44.],
        [ 91.,  87.,  65.],
        [ 91.,  88.,  64.]])
tensor([[104., 118.],
        [119., 133.],
        [ 57.,  69.],
        [ 80., 102.],
        [ 81., 101.]])


### In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

# nn.Linear

 we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [72]:
# define the model

model=nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3312,  0.3922, -0.4110],
        [ 0.0022, -0.0917,  0.3024]], requires_grad=True)
Parameter containing:
tensor([0.3966, 0.4248], requires_grad=True)


### PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [73]:
model.parameters()

<generator object Module.parameters at 0x7b1ec636cf90>

In [74]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3312,  0.3922, -0.4110],
         [ 0.0022, -0.0917,  0.3024]], requires_grad=True),
 Parameter containing:
 tensor([0.3966, 0.4248], requires_grad=True)]

# Generate predictions

In [75]:
pred=model(inputs)
pred

tensor([[-15.1748,   7.4416],
        [-21.5310,  11.9051],
        [  0.3015,   5.8634],
        [-31.7256,   7.8911],
        [-13.5737,  12.9382],
        [-15.8982,   7.5355],
        [-22.3342,  12.2992],
        [ -0.4407,   6.1680],
        [-31.0022,   7.7972],
        [-13.6536,  13.2384],
        [-15.9780,   7.8357],
        [-22.2543,  11.9990],
        [  1.1048,   5.4693],
        [-31.6457,   7.5908],
        [-12.8503,  12.8443]], grad_fn=<AddmmBackward0>)

## Loss Function
we can use the built-in loss function `mse_loss`.

The `nn.functional` package contains many useful loss functions and several other utilities.

In [76]:
import torch.nn.functional as F

In [77]:
loss_fn=F.mse_loss
loss=loss_fn(pred,targets)
print(loss)

tensor(8609.7031, grad_fn=<MseLossBackward0>)


## Optimizer

we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

In [78]:
# define the optimizer

opt=torch.optim.SGD(model.parameters(),lr=1e-5)

## Train the model

We are now ready to train the model.

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

we'll work batches of data .Let's define a utility function `fit` that trains the model for a given number of epochs.

In [83]:
# utility function to train the model

def fit(epochs,model,loss_fn,opt,train_dataloader):

  #repeat for given batches of data
  for i in range(epochs):

    # train with batches of data
    for inp,tar in train_dataloader:

      # 1. generate predictions
      pred=model(inp)

      # 2. calculate loss
      loss=loss_fn(pred,tar)

      # 3. compute gradients
      loss.backward()

      # 4. update parameters using the gradeints
      opt.step()

      # 5. reset the gradients to zero
      opt.zero_grad()
  # print the progress
      if (i+1) % 10 ==0:
        print('epoch [{} / {}], loss : {: .4f}'.format(i+1,epochs,loss.item()))





Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* updating parameters , we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 200 epochs.

In [85]:
fit(200, model, loss_fn, opt, train_dataloader)

epoch [10 / 200], loss :  15.4165
epoch [10 / 200], loss :  15.7699
epoch [10 / 200], loss :  7.2747
epoch [20 / 200], loss :  16.9982
epoch [20 / 200], loss :  10.4981
epoch [20 / 200], loss :  9.0331
epoch [30 / 200], loss :  11.4775
epoch [30 / 200], loss :  14.6703
epoch [30 / 200], loss :  10.0676
epoch [40 / 200], loss :  2.5477
epoch [40 / 200], loss :  15.1558
epoch [40 / 200], loss :  15.0295
epoch [50 / 200], loss :  7.1220
epoch [50 / 200], loss :  15.0359
epoch [50 / 200], loss :  9.0779
epoch [60 / 200], loss :  10.4192
epoch [60 / 200], loss :  8.0591
epoch [60 / 200], loss :  10.8952
epoch [70 / 200], loss :  9.0303
epoch [70 / 200], loss :  9.3491
epoch [70 / 200], loss :  10.6968
epoch [80 / 200], loss :  9.5827
epoch [80 / 200], loss :  6.5626
epoch [80 / 200], loss :  11.7162
epoch [90 / 200], loss :  9.5701
epoch [90 / 200], loss :  10.1149
epoch [90 / 200], loss :  4.8923
epoch [100 / 200], loss :  6.4080
epoch [100 / 200], loss :  8.4248
epoch [100 / 200], loss : 

In [86]:
# Let's generate predictions using our model and verify that they're close to our targets.
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.3414,  70.5709],
        [ 80.3825,  99.9811],
        [122.0816, 134.0178],
        [ 21.8995,  38.1067],
        [ 98.3830, 117.3923],
        [ 56.0535,  69.4775],
        [ 79.9086,  99.9699],
        [122.1923, 134.5574],
        [ 23.1874,  39.2001],
        [ 99.1971, 118.4746],
        [ 56.8675,  70.5597],
        [ 79.0946,  98.8876],
        [122.5555, 134.0290],
        [ 21.0855,  37.0245],
        [ 99.6709, 118.4858]], grad_fn=<AddmmBackward0>)

In [87]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [88]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.3555, 67.5508]], grad_fn=<AddmmBackward0>)

## The predicted yield of apples is 53.3 tons per hectare, and that of oranges is 67.5 tons per hectare.